## 2.1. 엘라스틱서치 기본 동작 빠르게 둘러보기
### 2.1.1. 문서 색인
* _id를 지정해 색인
  * _id 값은 문서를 인덱스에서 고유하게 식별하기 위한 값
  * 모든 문서는 _id 값을 가짐

In [4]:
%%bash
curl -X PUT --location "http://localhost:9200/my_index/_doc/1" \
    -H "Content-Type: application/json" \
    -d "{
          \"title\": \"hello world\",
          \"views\": 1234,
          \"public\": true,
          \"created\": \"2019-01-17T14:05:01.2342\"
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   285  100   141  100   144   3283   3353 --:--:-- --:--:-- --:--:--  8906


{"_index":"my_index","_id":"1","_version":2,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":1,"_primary_term":1}

* _id를 지정하지 않고 색인
  * 별도로 지정하지 않는 경우 자동으로 _id 값 생성
  * POST 메서드 사용

In [5]:
%%bash
curl -X POST --location "http://localhost:9200/my_index/_doc" \
    -H "Content-Type: application/json" \
    -d "{
          \"title\": \"hello world\",
          \"views\": 1234,
          \"public\": true,
          \"created\": \"2019-01-17T14:05:01.2342\"
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   304  100   160  100   144   3950   3555 --:--:-- --:--:-- --:--:--  9806


{"_index":"my_index","_id":"u2dXCI4B3xdUw5e3E9ub","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":2,"_primary_term":1}

### 2.1.2. 문서 조회
* 인덱스 이름과 _id 값 지정해 조회
* GET 메서드 사용
* _source 필드에서 문서 내용 확인 가능

In [6]:
%%bash
curl -X GET --location "http://localhost:9200/my_index/_doc/1"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   242  100   242    0     0   8321      0 --:--:-- --:--:-- --:--:-- 15125


{"_index":"my_index","_id":"1","_version":2,"_seq_no":1,"_primary_term":1,"found":true,"_source":{
          "title": "hello world",
          "views": 1234,
          "public": true,
          "created": "2019-01-17T14:05:01.2342"
        }}

### 2.1.3. 문서 업데이트
* 문서 색인 시 사용했던 API를 여러 번 호출할 경우 update 동작을 하는 것으로 보임
* 인덱스 이름과 _id 값 지정해 부분 업데이트

In [7]:
%%bash
curl -X POST --location "http://localhost:9200/my_index/_update/1" \
    -H "Content-Type: application/json" \
    -d "{
          \"doc\": {
            \"title\": \"hello elasticsearch!\"
          }
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   227  100   141  100    86   4556   2779 --:--:-- --:--:-- --:--:-- 12611


{"_index":"my_index","_id":"1","_version":3,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":3,"_primary_term":1}

### 2.1.4. 문서 검색
* 인덱스명 이후 `_search`를 붙인 후 GET 혹은 POST 메서드 사용
* 문서를 분석해 역색인을 만들어 두고 검색어를 분석해 유사도가 높은 문서를 찾음

In [9]:
%%bash
curl -X GET --location "http://localhost:9200/my_index/_search" \
    -H "Content-Type: application/json" \
    -d "{
          \"query\": {
            \"match\": {
              \"title\": \"hello world\"
            }
          }
        }"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   667    0   549  100   118  17006   3655 --:--:-- --:--:-- --:--:-- 26680


{"took":14,"timed_out":false,"_shards":{"total":1,"successful":1,"skipped":0,"failed":0},"hits":{"total":{"value":2,"relation":"eq"},"max_score":0.8754687,"hits":[{"_index":"my_index","_id":"u2dXCI4B3xdUw5e3E9ub","_score":0.8754687,"_source":{
          "title": "hello world",
          "views": 1234,
          "public": true,
          "created": "2019-01-17T14:05:01.2342"
        }},{"_index":"my_index","_id":"1","_score":0.18232156,"_source":{"title":"hello elasticsearch!","views":1234,"public":true,"created":"2019-01-17T14:05:01.2342"}}]}}

### 2.1.5. 문서 삭제
* 인덱스 이름과 _id 지정 후 DELETE 메서드 사용

In [10]:
%%bash
curl -X DELETE --location "http://localhost:9200/my_index/_doc/1"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   141  100   141    0     0   5200      0 --:--:-- --:--:-- --:--:--  7833


{"_index":"my_index","_id":"1","_version":4,"result":"deleted","_shards":{"total":2,"successful":1,"failed":0},"_seq_no":4,"_primary_term":1}

## 2.2. 엘라스틱서치 구조 개괄
* 문서 (document)
  * 저장과 색인의 대상이 되는 JSON 문서
* 인덱스
  * 문서를 모아두는 단위
  * 클라이언트 검색 요청의 단위
* 샤드
  * 인덱스의 내용을 분산해 저장
  * 고가용성을 위해 복제
    * 원복 역할을 주 샤드 (primary shard)라고 함
    * 복제본을 복제본 샤드 (replication shard)라고 함
* _id
  * 문서의 고유 구분자 
  * 인덱스 이름, _id와 라우팅 조합은 클러스터 내 고유
* 타입
  * 인덱스 안의 여러 문서를 묶어 논리적 단위로 나눔
  * 6 버전부터 인덱스 하나에 하나의 타입만 둘 수 있도록 제한
  * 7 버전부터 API에서 타입 명시하는 부분이 중단되고, 기본값인 `_doc` 들어감
* 노드
  * 프로세스 하나가 노드 하나를 구성
  * 여러 개의 샤드를 가짐
    * 고가용성을 위해 같은 종류의 노드를 가지지 않음
  * 데이터 노드, 마스터 노드, 조정 노드 등 역할 중 하나 이상의 역할을 수행
    * 데이터 노드
      * 샤드를 보유하고 샤드에 읽기, 쓰기 작업을 수행
    * 마스터 노드
      * 클러스터 관리
      * 후보 노드 중 한 대가 선출
    * 조정 노드
      * 클라이언트 요청을 받아 데이터 노드에 분배하고 클라이언트에 응답을 돌려줌
* 클러스터
  * 여러 개의 노드가 하나의 클러스터 구성
  
## 2.3. 엘라스틱서치 내부 구조와 루씬
* 루씬
  * 문서를 색인하고 검색하는 라이브러리
  * 엘라스틱서치의 코어 라이브러리


### 2.3.1. 루씬 flush
* 메모리에 들고 있는, 문서 색인, 업데이트, 삭제 등의 변경을 디스크에 flush
* 색인된 정보는 파일로 저장
  * 엘라스틱서치 내부적으로 루씬의 DirectoryReader 클래스를 이용해 파일을 열고, 색인에 접근할 수 있는 IndexReader 객체를 얻음
* 파일 연 시점 기준으로 검색해 검색 결과에 변경 내용을 반영하기 위해서는 새로 파일을 열어야 함
  * 엘라스틱서치 refresh
  * 엘라스틱서치 내부적으로 DirectoryReader.openIfChanged 호출해 변경이 적용된 IndexReader를 연 후 기존 IndexReader를 닫음
  * 비용이 있는 작업이기 때문에 일정 주기마다 실행되거나 명시적으로 실행

### 2.3.2. 루씬 commit
* 루씬 flush는 디스크에 파일이 안전하게 기록되는 것까지 보장하지 않음
* fsync 시스템콜을 통해 주기적으로 커널 시스템의 페이지 캐시 내용과 실제 디스크에 기록된 내용 싱크를 맞춤
  * fsync
    * 변경 파일 (data와 metadata)를 하드 디스크에 저장되도록 요청하며, 저장이 될 때까지 기다림
    * [참고 자료: fsync-sync-차이점-활용법](https://backuporigin.tistory.com/entry/fsync-sync-%EC%B0%A8%EC%9D%B4%EC%A0%90-%ED%99%9C%EC%9A%A9%EB%B2%95)
* 엘라스틱서치 flush는 내부적으로 루씬 commit을 거침
  * 엘라스틱서치 refresh보다 비용이 큼
  * 일정 주기마다 실행되거나 명시적으로 실행
* fsync 중 하드웨어 캐시를 사용하는 환경에서 하드웨어 실패가 나는 상황까지 정합성을 보장하진 않음

### 2.3.3. 세그먼트
* 디스크에 기록된 파일들의 모음
* 루씬의 검색 대상
  * 개수를 무작정 늘릴 수 없기 때문에 병합 수행
    * 병합 수행 시 삭제 플래그 표시된 데이터를 실제로 지움
    * 병합 시 검색 성능 향상 기대할 수 있음
    * forcemerge API를 통해 명시적으로 세그먼트 병합 수행 가능
      * 추가 색인이 없을 것이 보장될 때 수행 권장
      * 5GB 이상의 큰 세그먼트가 존재하는 상황에서 데이터 변경 시 추가된 작은 세그먼트는 세그먼트 병합 대상에서 영원히 누락될 수 있기 때문
* 불변 데이터
  * 새 문서가 들어올 경우 새 세그먼트 생성
  * 기존 문서 삭제 시 삭제 플래그만 표시
  * 기존 문서 수정 시 삭제 플래그 표시 후 새 세그먼트 생성
  
### 2.3.4. 루씬 인덱스와 엘라스틱서치 인덱스 
* 루씬 인덱스
  * 여러 세그먼트의 모음
  * 루씬은 인덱스 안에서만 검색 가능
  * 이를 래핑한 게 엘라스틱서치의 샤드
* 엘라스틱서치 인덱스
  * 여러 샤드의 모음
  * 새 문서가 들어오면 라우팅해 여러 샤드에 분산시켜 색인 및 저장
  * 검색 요청이 들어온 경우 샤드 대상으로 분산 검색한 후 그 결과를 모아 최종 응답
![elasticsearch index structure](../assets/images/2-1.elasticsearch-index-structure.png)

### 2.3.5. translog
* 장애 발생 시 commit되지 않은 데이터가 유실되는 것을 방지하기 위해 남기는 작업 로그
* 색인, 삭제 작업이 루씬 인덱스에 수행된 직후 기록
  * translog 기록까지 완료되어야 성공으로 승인
* 장애 발생 시 샤드 복구 단계에서 translog 읽음
* 엘라스틱서치 flush 수행 시 루씬 commit 수행하고 새로운 translog를 만듦
* 백그라운드에서 엘라스틱서치 flush가 수행되며 translog의 크기를 적정 수준으로 유지
* 디스크에 fsync된 데이터만 보존
* `index.translog.durability` 설정 `async`로 지정할 경우 `index.translog.sync_interval` (기본값 5s) 주기에 의해서만 translog fsync 수행